In [6]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import StaticAdjective, PointerAdjective, PointerRankingAdjective
from src.explainer.explanation import AdjectiveExplanation, AssumptionExplanation, ConditionalExplanation, CompositeExplanation

In [4]:
# Example usage:
class MinMaxNode:
    def __init__(self, *, score=None, is_leaf, children=None, score_child=None):
        
        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.is_leaf = is_leaf
        self.children = children or []
        self.parent = None
        
        
        if len(self.children) > 0:
            for child in children:
                child.parent=self

# Create a simple game tree
leaf1 = MinMaxNode(score=3, is_leaf=True)
leaf2 = MinMaxNode(score=5, is_leaf=True)
root = MinMaxNode(is_leaf=False, children=[leaf1, leaf2], score_child=leaf2)

In [11]:
framework = ArgumentationFramework()

# Adding adjectives
framework.add_adjective(StaticAdjective("is_leaf", lambda node: node.is_leaf))
framework.add_adjective(PointerRankingAdjective("better", "score", lambda x, y: x.score > y.score))

# Renaming automatically created adjectives
framework.rename_adjective("max_better", "maxoptimal")
framework.rename_adjective("min_better", "minoptimal")

# Setting tree search motivation
framework.set_tree_search_motivation("maxoptimal")

framework.set_adjective_explanation("score", 
    ConditionalExplanation(
    condition = lambda node: node.is_leaf,
    true_explanation = AssumptionExplanation("Leaf nodes have scores from the evaluation function"),
    false_explanation = CompositeExplanation(
        AssumptionExplanation("Internal nodes have scores from children"),
        AdjectiveExplanation("minoptimal", "min_score_child")
    )
))

framework.add_adjective(
    PointerAdjective("score_child",
    #lambda node: min(node.children, key=lambda child: child.score),
    getter = lambda node: node.score_child,
    explanation = ConditionalExplanation(
        condition=lambda node: framework.get_adjective("minoptimal").evaluate(node),
        true_explanation=AdjectiveExplanation("minoptimal", "min_score_child"),
        false_explanation=AdjectiveExplanation("maxoptimal", "min_score_child")
    )
))

In [13]:
explainer = ArgumentativeExplainer(framework)

Try

In [14]:
# Generate explanations
print(explainer.explain_adjective(root.children[1], "maxoptimal"))


max_better is defined by its condition


In [ ]:
print(explainer.query_explanation(root, "Why is child 1 maxoptimal?"))

((Leaf nodes have scores from the evaluation function) → maxoptimal(True))
